###### Prelude

In [22]:
# The GPU Devotes More Transistors to Data Processing than CPU

<img src="img/gpu_1.png" width="360" height="360">

At its core are three key abstractions - a hierarchy of thread groups, shared memories, and barrier synchronization - that are simply exposed to the programmer as a minimal set of language extensions. 

compiled CUDA program can execute on any number of multiprocessors as illustrated by Figure 5, and only the runtime system needs to know the physical multiprocessor count. 

<img src="img/automatic-scalability.png" width="360" height="360">

<b>Note</b>: A GPU is built around an array of <b>Streaming Multiprocessors (SMs) </b> (see Hardware Implementation for more details). A multithreaded program is partitioned into blocks of threads that execute independently from each other, so that a GPU with more multiprocessors will automatically execute the program in less time than a GPU with fewer multiprocessors. 

<b>Threads</b>

For convenience, <b>threadIdx</b> is a 3-component vector, so that threads can be identified using a one-dimensional, two-dimensional, or three-dimensional thread index, <b>forming a one-dimensional, two-dimensional, or three-dimensional block of threads</b>, called a <b>thread block</b>. This provides a natural way to invoke computation across the elements in a domain such as a vector, matrix, or volume. 

The index of a thread and its thread ID relate to each other in a straightforward way: For a one-dimensional <b>block</b>, they are the same; for a two-dimensional block of size (Dx, Dy),the thread ID of a thread of index (x, y) is (x + y Dx); for a three-dimensional block of size (Dx, Dy, Dz), the thread ID of a thread of index (x, y, z) is (x + y Dx + z Dx Dy). 

There is a limit to the number of threads per block, since all threads of a block are expected to reside on the same processor core and must share the limited memory resources of that core. <b>On current GPUs, a thread block may contain up to 1024 threads?</b>. 

However, a <b>kernel</b> can be executed by multiple <b>equally-shaped</b> thread blocks,
so that

<b>the total number of threads is equal to the number of threads per block times the number of blocks</b>. 

Blocks are organized into a one-dimensional, two-dimensional, or three-dimensional grid of thread blocks as illustrated by Figure 6. The number of thread blocks in a grid is usually dictated by the size of the data being processed or the number of processors in the system, which it can greatly exceed. 

<img src="img/grid-of-thread-blocks.png" width="360" height="360">

Each block within the grid can be <b>identified</b> by a one-dimensional, two-dimensional, or three-dimensional index accessible within the kernel through the built-in blockIdx variable. The dimension of the <b>thread block</b> is accessible within the kernel through the built-in <b>blockDim</b> variable

The grid is created with enough blocks to have one thread per matrix element as before.

<img src="img/memory-hierarchy.png" width="360" height="360">

<b>Heterogeneous programming</b>

<img src="img/heterogeneous-programming.png" width="360" height="360">

###### The CUDA Programming Model

Ufuncs (and generalized ufuncs mentioned in the bonus notebook at the end of the tutorial) are the easiest way in Numba to use the GPU, and present an abstraction that requires minimal understanding of the CUDA programming model. However, not all functions can be written as ufuncs. Many problems require greater flexibility, in which case you want to write a CUDA kernel, the topic of this notebook.

Fully explaining the CUDA programming model is beyond the scope of this tutorial. We highly recommend that everyone writing CUDA kernels with Numba take the time to read Chapters 1 and 2 of the CUDA C Programming Guide:

    Introduction: http://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#introduction
    Programming Model: http://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#programming-model

The programming model chapter gets a little in to C specifics, but familiarity with CUDA C can help write better CUDA kernels in Python.

We will be writing a kernel that decribes the execution of a <b>single thread</b> in this hierarchy. The CUDA compiler and driver will execute our kernel across a thread grid that is divided into blocks of threads. <b>Threads within the same block can exchange data very easily during the execution of a kernel</b>, whereas threads in different blocks should generally not communicate with each other (with a few exceptions).

Deciding the best size for the CUDA thread grid is a complex problem (and depends on both the algorithm and the specific GPU compute capability), but here are some very rough heuristics that we follow:

- the <b>size of a block</b> should be a <b>multiple of 32 threads</b>, with typical block sizes between 128 and 512 threads per block.
- <b>the size of the grid</b> should ensure the <b>full GPU is utilized</b> where possible. <b>Launching a grid where the number of blocks is 2x-4x the number of "multiprocessors" on the GPU is a good starting place</b>. <b>Something in the range of 20 - 100 blocks is usually a good starting point.</b>
- The CUDA kernel launch overhead does depend on the <b>number of blocks</b>, so we find it best not to launch a grid where <b>the number of threads equals the number of input elements</b> when the input size is very big. <b>We'll show a pattern for dealing with large inputs below.</b>

Each thread distinguishes itself from the other threads using its unique thread (threadIdx) and block (blockIdx) index values, which can be multidimensional if launched that way.

###### Cuda Kernels

In [37]:
from numba import cuda
import numpy as np

@cuda.jit
def add_kernel(x, y, out):
    tx = cuda.threadIdx.x # this is the unique thread ID within a 1D block
    ty = cuda.blockIdx.x  # Similarly, this is the unique block ID within the 1D grid

    block_size = cuda.blockDim.x  # number of threads per block
    grid_size = cuda.gridDim.x    # number of blocks in the grid
    
    start = tx + ty * block_size # why not tx * block_size?
    stride = block_size * grid_size 
    
    print("tx:", tx) # How to print?
    
    # assuming x and y inputs are same length
    for i in range(start, x.shape[0], stride):
        out[i] = x[i] + y[i]

In [42]:
# How to get ThreadID?
# How to understand how data is distributed

That's a lot more typing than our ufunc example, and it is much more limited: only works on 1D arrays, doesn't verify input sizes match, etc. Most of the function is spent figuring out how to turn the block and grid indices and dimensions into unique offsets into the input arrays. The pattern of computing a starting index and a stride is a common way to ensure that your grid size is independent of the input size. The striding will maximize bandwidth by ensuring that threads with consecuitive indices are accessing consecutive memory locations as much as possible.
<b> Thread indices beyond the length of the input (x.shape[0], since x is a NumPy array) automatically skip over the for loop</b>.

Also note that we did not need to specify a type signature for the CUDA kernel. Unlike @vectorize, Numba can infer the type signature from the inputs automatically, and much more reliably.

Let's call the function now on some data:

In [43]:
n = 100000
x = np.arange(n).astype(np.float32)
y = 2 * x
out = np.empty_like(x)

threads_per_block = 128
blocks_per_grid = 30

add_kernel[blocks_per_grid, threads_per_block](x, y, out)
print(out[:10])
print(out.shape)

[ 0.  3.  6.  9. 12. 15. 18. 21. 24. 27.]
(100000,)


The unusual syntax for calling the kernel function is designed to mimic the CUDA Runtime API in C, where the above call would look like:

<b>add_kernel<<<blocks_per_grid, threads_per_block>>>(x, y, out)</b>

The arguments within the square brackets define the size and shape of the thread grid, and the arguments with parentheses correspond to the kernel function arguments.

Note that, unlike the ufunc, the arguments are passed to the kernel as full NumPy arrays. The kernel can access any element in the array it wants, regardless of its position in the thread grid. This is why CUDA kernels are significantly more powerful that ufuncs. (But with great power, comes a greater amount of typing...)

Numba includes several helper functions to simplify the thread offset calculations above. You can write the function much more simply as:

<b>numba.pydata.org/numba-doc/dev/cuda/kernels.html#absolute-positions</b>

In [61]:
@cuda.jit
def add_kernel(x, y, out):
    start = cuda.grid(1)      # 1 = one dimensional thread grid, returns a single value
    stride = cuda.gridsize(1) # ditto

    # assuming x and y inputs are same length
    for i in range(start, x.shape[0], stride):
        out[i] = x[i] + y[i]

###### <b>Example incrementation from numba doc</b>

###### Next part

As before, using NumPy arrays forces Numba to allocate GPU memory, copy the arguments to the GPU, run the kernel, then copy the argument arrays back to the host. This not very efficient, so you will often want to allocate device arrays:

In [64]:
x_device = cuda.to_device(x)
y_device = cuda.to_device(y)
out_device = cuda.device_array_like(x)

In [65]:
%timeit add_kernel[blocks_per_grid, threads_per_block](x, y, out)

2.06 ms ± 342 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [66]:
%timeit add_kernel[blocks_per_grid, threads_per_block](x_device, y_device, out_device); out_device.copy_to_host()

440 µs ± 56.4 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


###### Kernel Synchronization

One extremely important caveat should be mentioned here: <b>CUDA kernel execution is designed to be asynchronous with respect to the host program</b>. This means that the kernel launch (add_kernel[blocks_per_grid, threads_per_block](x_device, y_device, out_device)) <b>returns immediately, allowing the CPU to continue executing while the GPU works in the background</b>. Only host<->device memory copies or an explicit synchronization call will force the CPU to wait until previously queued CUDA kernels are complete.

When you pass host NumPy arrays to a CUDA kernel, <b>Numba has to synchronize on your behalf</b>, but if you pass device arrays, processing will continue. If you launch multiple kernels in sequence without any synchronization in between, they will be queued up to run sequentially by the driver, which is usually what you want. If you want to run multiple kernels on the GPU in parallel (sometimes a good idea, but beware of race conditions!), take a look at <b>CUDA streams</b>.

Here's some sample timings (using %time, which only runs the statement once to ensure our measurement isn't affected by the finite depth of the CUDA kernel queue):

In [71]:
# CPU input/output arrays, implied synchronization for memory copies
%time add_kernel[blocks_per_grid, threads_per_block](x, y, out)

Wall time: 2.99 ms


<b>cuda.synchronize???</b>

In [72]:
# GPU input/output arrays, no synchronization (but force sync before and after)
cuda.synchronize() #?
%time add_kernel[blocks_per_grid, threads_per_block](x_device, y_device, out_device)
cuda.synchronize()

Wall time: 997 µs


In [74]:
# GPU input/output arrays, include explicit synchronization in timing
cuda.synchronize()
%time add_kernel[blocks_per_grid, threads_per_block](x_device, y_device, out_device); cuda.synchronize()

Wall time: 97.7 ms


<b>Always be sure to synchronize with the GPU when benchmarking CUDA kernels!</b>

###### Atomic Operations and Avoiding Race Conditions

CUDA, like many general purpose parallel execution frameworks, makes it possible to have <b>race condtions</b> in your code. A race condition in CUDA arises when <b>threads read or write a memory location that might be modified by another independent thread</b>. Generally speaking, you need to worry about:

- read-after-write hazards: One thread is reading a memory location at the same time another thread might be writing to it.
- write-after-write hazards: Two threads are writing to the same memory location, and only one write will be visible when the kernel is complete.

A common strategy to avoid both of these hazards is to organize your CUDA kernel algorithm such that <b>each thread has exclusive responsibility for unique subsets of output array elements</b>, <b>and/or to never use the same array for both input and output in a single kernel call</b>. (Iterative algorithms can use a double-buffering strategy if needed, and switch input and output arrays on each iteration.)

However, there are many cases where different threads need to combine results. Consider something very simple, like: "every thread increments a global counter." Implementing this in your kernel requires each thread to:

- Read the current value of a global counter.
- Compute counter + 1.
- Write that value back to global memory.

However, there is no guarantee that another thread has not changed the global counter between steps 1 and 3. To resolve this problem, CUDA provides <b>"atomic operations"</b> which will <b>read, modify and update a memory location in one, indivisible step</b>. Numba supports several of these functions, described here.

Let's make our thread counter kernel:

In [78]:
@cuda.jit
def thread_counter_race_condition(global_counter):
    global_counter[0] += 1  # This is bad
    
@cuda.jit
def thread_counter_safe(global_counter):
    cuda.atomic.add(global_counter, 0, 1)  # Safely add 1 to offset 0 in global_counter array

In [80]:
# This gets the wrong answer
global_counter = cuda.to_device(np.array([0], dtype=np.int32))
thread_counter_race_condition[64, 64](global_counter)

print('Should be %d:' % (64*64), global_counter.copy_to_host())

Should be 4096: [1]


In [81]:
# This works correctly
global_counter = cuda.to_device(np.array([0], dtype=np.int32))
thread_counter_safe[64, 64](global_counter)

print('Should be %d:' % (64*64), global_counter.copy_to_host())

Should be 4096: [4096]


###### Exercise 

For this exercise, create a histogramming kernel. This will take an array of input data, a range and a number of bins, and count how many of the input data elements land in each bin. Below is an example CPU implementation of histogramming:

In [82]:
def cpu_histogram(x, xmin, xmax, histogram_out):
    '''Increment bin counts in histogram_out, given histogram range [xmin, xmax).'''
    # Note that we don't have to pass in nbins explicitly, because the size of histogram_out determines it
    nbins = histogram_out.shape[0]
    bin_width = (xmax - xmin) / nbins
    
    # This is a very slow way to do this with NumPy, but looks similar to what you will do on the GPU
    for element in x:
        bin_number = np.int32((element - xmin)/bin_width)
        if bin_number >= 0 and bin_number < histogram_out.shape[0]:
            # only increment if in range
            histogram_out[bin_number] += 1

In [83]:
x = np.random.normal(size=10000, loc=0, scale=1).astype(np.float32)
xmin = np.float32(-4.0)
xmax = np.float32(4.0)
histogram_out = np.zeros(shape=10, dtype=np.int32)

cpu_histogram(x, xmin, xmax, histogram_out)

histogram_out

array([   6,   84,  430, 1609, 2883, 2924, 1496,  488,   77,    3])

###### cuda implementation

In [ ]:
@cuda.jit
def cuda_histogram(x, xmin, xmax, histogram_out):
    '''Increment bin counts in histogram_out, given histogram range [xmin, xmax).'''
    nbins = histogram_out.shape[0]
    bin_width = (xmax - xmin) / nbins
    
    

Do it!

Atomic example

In [89]:
@cuda.jit
def max_example(result, values):
    """Find the maximum value in values and store in result[0]"""
    tid = cuda.threadIdx.x
    bid = cuda.blockIdx.x
    bdim = cuda.blockDim.x
    i = (bid * bdim) + tid
    cuda.atomic.max(result, 0, values[i])


arr = np.random.rand(16384)
result = np.zeros(1, dtype=np.float64)

max_example[256,64](result, arr)
print(result[0]) # Found using cuda.atomic.max
print(max(arr))  # Print max(arr) for comparision (should be equal!)

0.9998447927977847
0.9998447927977847


In [90]:
@cuda.jit
def max_example_3d(result, values):
    """
    Find the maximum value in values and store in result[0].
    Both result and values are 3d arrays.
    """
    i, j, k = cuda.grid(3)
    # Atomically store to result[0,1,2] from values[i, j, k]
    cuda.atomic.max(result, (0, 1, 2), values[i, j, k])

arr = np.random.rand(1000).reshape(10,10,10)
result = np.zeros((3, 3, 3), dtype=np.float64)
max_example_3d[(2, 2, 2), (5, 5, 5)](result, arr)
print(result[0, 1, 2], '==', np.max(arr))

0.999790882371729 == 0.999790882371729
